# PCAF Data Pipeline to ingest the data provided by EDGAR (fossil CO2 values)

The source excel file contains 5 sheets and parsed&transposed using the configuration data saved in the file "Edgar.ini".

The output first saved in a CSV file and afterwards copied in a dataframe and saved in a database table containing the fields "provider,attribute,value,unit".


In [ ]:
# 'capture' magic prevents long outputs from spamming your notebook
#%%capture pipoutput

# For loading predefined environment variables from files
# Typically used to load sensitive access credentials
%pip install python-dotenv

# Standard python package for interacting with S3 buckets
%pip install boto3

# Interacting with Trino and using Trino with sqlalchemy
%pip install trino sqlalchemy sqlalchemy-trino

# Pandas and parquet file i/o
%pip install pandas pyarrow fastparquet

# OS-Climate utilities to make data ingest easier
%pip install osc-ingest-tools
# install pycountry to retrieve ISO codes
%pip install jedi==0.17.2
%pip install -U "ipython>=7.20"
%pip install pycountry
%pip install country-converter



In [ ]:
from dotenv import dotenv_values, load_dotenv
import osc_ingest_trino as osc
import os
import pathlib

### Load Environment Variables

In [ ]:
dotenv_dir = os.environ.get('CREDENTIAL_DOTENV_DIR', os.environ.get('PWD', '/opt/app-root/src'))
dotenv_path = pathlib.Path(dotenv_dir) / 'credentials.env'
if os.path.exists(dotenv_path):
    load_dotenv(dotenv_path=dotenv_path,override=True)

In [ ]:
# use a catalog that is configured for iceberg
ingest_catalog = 'osc_datacommons_iceberg_dev'
ingest_schema = 'sandbox'
ingest_table = 'pcaf_sovereign_edgar'

In [ ]:
import trino
from sqlalchemy.engine import create_engine

env_var_prefix = 'TRINO'

sqlstring = 'trino://{user}@{host}:{port}/'.format(
    user = os.environ[f'{env_var_prefix}_USER'],
    host = os.environ[f'{env_var_prefix}_HOST'],
    port = os.environ[f'{env_var_prefix}_PORT']
)
sqlargs = {
    'auth': trino.auth.JWTAuthentication(os.environ[f'{env_var_prefix}_PASSWD']),
    'http_scheme': 'https',
    'catalog': 'osc_datacommons_iceberg_dev'
}
engine = create_engine(sqlstring, connect_args = sqlargs)
connection = engine.connect()

trino_bucket = osc.attach_s3_bucket("S3_DEV")

In [ ]:
import boto3

s3_source = boto3.resource(
    service_name="s3",
    endpoint_url=os.environ['S3_LANDING_ENDPOINT'],
    aws_access_key_id=os.environ['S3_LANDING_ACCESS_KEY'],
    aws_secret_access_key=os.environ['S3_LANDING_SECRET_KEY'],
)
source_bucket = s3_source.Bucket(os.environ['S3_LANDING_BUCKET'])

In [ ]:
Open a Trino connection using JWT for authentication

In [ ]:
# make sure schema exists, or table creation below will fail in weird ways
sql = f"""
create schema if not exists {ingest_catalog}.{ingest_schema}
"""
qres = engine.execute(sql)
print(qres.fetchall())

In [ ]:
# Show available schemas to ensure trino connection is set correctly
schema_read = engine.execute(f'show schemas in {ingest_catalog}')
for row in schema_read.fetchall():
    print(row)

In [ ]:
Load GDP file (updated sporadically from https://data.worldbank.org/indicator/NY.GDP.PCAP.PP.CD)

In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

ticker_file = s3_source.Object(os.environ['S3_LANDING_BUCKET'],'PCAF-sovereign-footprint/EDGAR/EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls')
ticker_file.download_file(f'/tmp/EDGARv6.0_FT2020_fossil_CO2_GHG_booklet2021.xls')
%run TransposeXLS.py --config Edgar.ini --output=EDGAR.csv 
df = pd.read_csv('EDGAR.csv')
df= df[['rec_source','data_provider','country_iso_code','country_name','validity_date','attribute','value','unit']].dropna(subset=['value'])
#df["country_iso_code"]='###'
#df.validity_date = df.validity_date.astype(str)
df = df.convert_dtypes()
print(df.info(verbose=True))
df
#df


In [ ]:
# EDGAR does not deliver ISO codes , set the field using pycountry


import pycountry  as c
import country_converter as coco

def getCountryISOCode(country_name):
    try:
        country=c.countries.search_fuzzy(country_name)
        if country is None:
            print(country_name)
            return "###"
        else:
            return country[0].alpha_3
    except:
        print(country_name);
        return "###"

    
cc = coco.CountryConverter()

#some_names = ['Democratic Republic of the Congo']
#some_names = ['Germany']
df["country_iso_code"] = cc.convert(names = df["country_name"], to = 'ISO3')
df
    
#df["country_conv_name"] = df["country_name"].apply(lambda x: getCountryISOCode(x))

#c.countries.search_fuzzy('Democratic Republic of the Congo')   
#c.countries.search_fuzzy('Congo')   

#c.countries.search_fuzzy('Germany')   

    
#df["country_iso_code"] = df["country_name"].apply(lambda x: getCountryISOCode(x))
#df






In [ ]:
import osc_ingest_trino as osc
columnschema = osc.create_table_schema_pairs(df) 

sql = f"""
drop table if exists {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
print(sql)
qres = engine.execute(sql)
print(qres.fetchall())


In [ ]:
tabledef = f"""
create table if not exists {ingest_catalog}.{ingest_schema}.{ingest_table}(
{columnschema}
) with (
    format = 'ORC',
    partitioning = array['validity_date']
)
"""
print(tabledef)
qres = engine.execute(tabledef)
print(qres.fetchall())

In [ ]:
# Delete all data from our db, so we start with empty table
sql=f"""
delete from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
qres = engine.execute(sql)
print(qres.fetchall())

In [ ]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}
"""
pd.read_sql(sql, engine)


In [ ]:
print(ingest_catalog)
#df=df.drop(df[df.country_name=="cote d'ivoire"].index)
#df["country_iso_code"]='###'
#df.validity_date = df.validity_date.astype(str)
df.to_sql(ingest_table,
           con=engine, 
           schema=ingest_schema,
           if_exists='append',
           index=False,
           method=osc.TrinoBatchInsert(batch_size = 5000, verbose = True))

In [ ]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.{ingest_table}" + "where validity_date=2020 and attribute='fossil_CO2_totals_by_country'"""
pd.read_sql(sql, engine)
#attribute='fossil_CO2_totals_by_country'
